In [1]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/balbi/Downloads/pyMorfologik-master'))
if module_path not in sys.path:
    sys.path.append(module_path)
from pymorfologik import Morfologik
from pymorfologik.parsing import ListParser
from pymorfologik.parsing import BaseParser
from pymorfologik.parsing import DictParser

import numpy as np
from __future__ import print_function
import string
from multiprocessing import cpu_count, Pool
import dask.dataframe as dd
from dask.multiprocessing import get
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
def getEmotionStatusOfText(result):
    if(result < 0):
        return 'negative'
    elif(result > 0):
        return 'positive'
    else:
        return 'neutral'

Series([], Name: word, dtype: object)

In [ ]:
def lemmatisation_dictParser(text):
    parser = DictParser()
    stemmer = Morfologik()
    stemming = stemmer.stem([text], parser)
    words_list = list()
    for key, val in stemming.items():
        words_list.append(val[0].encode('utf8'))
    return words_list

In [ ]:
emotions = pd.read_csv('emotionsTable-with-neutralEmotion.csv')
def detectingEmotionsInText(words_list):
    numberOfWords = len(words_list)
    numberOfWordsFound = 0
    result = 0
    if(numberOfWords > 0):
        emotionsCounter = 0
        for el in words_list: 
            if((emotions['word'] == el)).any():
                print(el)
                emotionsCounter += float(emotions.loc[emotions['word'] == el]['sentiment'].mean())  
                numberOfWordsFound = numberOfWordsFound + 1
        result = emotionsCounter/numberOfWords
    return [result, numberOfWordsFound]

In [ ]:
%%time
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')

result = pd.read_csv('mediaKrytyk-commentsRates.csv')
result = result.reset_index()

def calculateOneRow(index,comment, emotion, rate):
    lemmatisation_text = re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']", '', comment)
    text = lemmatisation_dictParser(lemmatisation_text)
    [result, number] = detectingEmotionsInText(text)
    result_emotion = getEmotionStatusOfText(result)
    return {"text": comment, 'words': len(comment.split()), 'wordsFound': number, "resultOfDetecting": result, "detectedEmotion": result_emotion, 'emotion': emotion, 'rate': rate}

def parallelize(data, func):
    print (partitions)
    data_split = np.array_split(data, partitions)
    pool = Pool(cores)
    mapResult = pool.map(func, data_split)
    data = pd.concat(mapResult)
    pool.close()
    pool.join()
    return data

cores = cpu_count()
partitions = cores

ddata = dd.from_pandas(result, npartitions=partitions)

result = ddata.map_partitions(lambda df: df.apply((lambda row: calculateOneRow(*row)), axis=1)).compute(get=get)
result_df = pd.DataFrame()
for o in result:
    result_df = result_df.append(o, ignore_index=True)
    
result_df.to_csv('mediaKrytyk-comments-analysis-dictParser.csv', encoding='utf-8', index=False) 